## ZAF Quantification model

In [2]:
import math
import pandas as pd
# Fluorescence of Fe K alpha in 10wt% Fe-90wt% Ni Alloy
elements = ["Ni", "Fe"]
mac_Ni_K = [58.9, 379.6] # Ni Emitter
mac_Fe_K = [90,71.4] # Fe Emitter
omega = [0.37, False]
take_off_angle = 52.5
A = [58.71,55.847]
Z = [28, 26]
Ec = [8.332, 7.11] # keV
E0 = 30
C = [0.9, 0.1]

In [11]:
#### Ai Absorption ####
### X = mac * csc(psi)
X_Ni = mac_Ni_K[0] * (1/math.sin(math.radians(take_off_angle)))
print('X_Ni: ', X_Ni)
X_Fe = mac_Ni_K[1] * (1/math.sin(math.radians(take_off_angle)))
print('X_Fe: ', X_Fe)
X_tot = C[0] * X_Ni + C[1] * X_Fe
print("X_tot", X_tot)
### sigma
sigma_Ni = (4.5*pow(10,5))/(pow(E0, 1.65)-pow(Ec[0], 1.65))
sigma_Fe = (4.5*pow(10,5))/(pow(E0, 1.65)-pow(Ec[1], 1.65))
sigma_tot = C[0] * sigma_Ni + C[1] * sigma_Fe
print("sigma_tot", sigma_tot)
### h = 1.2 * A/Z^2
h_Ni = 1.2 * A[0]/pow(Z[0],2)
h_Fe = 1.2 * A[1]/pow(Z[1],2)
h_tot = C[0] * h_Ni + C[1] * h_Fe
print("h_tot", h_tot)
### 1/f(X)
reverse_f_X = (1 + X_tot/sigma_tot)*(1 + (h_tot/(1+h_tot))*X_tot/sigma_tot)
print(1/reverse_f_X)


X_Ni:  74.24182518520458
X_Fe:  478.47532835829645
X_tot 438.0519780409873
sigma_tot 7388.332548300779
h_tot 0.09820924224127521
0.9390498998838342
[+] The program is considering the emission of  Ni Ka
  element          X
0      Ni   73.10740
1      Fe  433.60251


In [89]:
#### Fluorescence F ####
# correction negligible if (E-Ec) > 5keV
# The mean depth of production of fluorescence radiation is greater than that of primary radiation (electrons)
# The correction factor Ifij/Ii relates the intensity of radiation of element i produced by fluorescence by element j, Ifij, 
# to the electron-generated intensity of radiation from element i, Ii;.
# REED (1965)
# absorption jump ratio
# (r_i - 1)/r_i = 0.88 for K line and equal 0.75 for L line 
abs_jump_ratio = {'K':0.88, 'L': 0.75}
# Y_0 = 0.5 * ((r_i - 1)/r_i) * w_j * (A_i/A_j)
## Fe
Y_0 = 0.5 * (abs_jump_ratio['K'] * omega[0] * (A[1]/A[0]))

Ui = E0/Ec[1]
Uj = E0/Ec[0]
Y_1 = pow(((Uj - 1)/(Ui - 1)),1.67)

# mac_ij is the mass absorption coefficient of element i for radiation from element j
# mac_j_spec is the mass absorption coefficient of the specimen for radiation from element j
Y_2 = mac_Ni_K[1]/(C[0]*mac_Ni_K[0] + C[1]*mac_Ni_K[1])

# Y3 account for absorption
u = ((C[0] * mac_Fe_K[0] + C[1] * mac_Fe_K[1])/(C[0] * mac_Ni_K[0] + C[1] * mac_Ni_K[1])) * (1/math.sin(math.radians(take_off_angle)))

# Ec is evaluated for element i
v = (3.3 * pow(10,5))/((pow(E0,1.65) - pow(Ec[1],1.65))*(C[0] * mac_Ni_K[0] + C[1] * mac_Ni_K[1]))
# Pij factor for the type of fluorescence occuring, if KK ( a K line fluoresces a K line) or LL fluorescence occurs Pij=1; KL or LK Pij = 4.76 for LK and 0.24 for KL
Pij = 1
Y_3 = (math.log(1 + u)/u) + (math.log(1+v)/v)
# in case the standard is a pure element
Ifij_by_Ii = C[0]*Y_0*Y_1*Y_2*Y_3*Pij
Fi = 1/(1 + Ifij_by_Ii)
print("E0, take off angle,", E0,",",take_off_angle)
print("If_Fe_Ni/I_Fe, ", Ifij_by_Ii)
print("Fi, ", Fi)

E0, take off angle, 15 , 52.5
If_Fe_Ni/I_Fe,  0.01047216365642225
Fi,  0.9896363660147466


In [56]:
#### Test data
elements = ["Al","Cu"]
C = [.02, .98]
Z = [13,29]
A = [26.98, 63.55]
Ec = [1.56, 8.98]
E0 = 15
mac_Cu_K = [49.6, 53.7]
mac_Al_K = [385.7, 5377]
take_off_angle =  52.5
#### Atomic Number Factor Z ####
# The so-called atomic number effect in electron microprobe analysis arises from two phenomena, namely, electron backscattering and electron retardation.
# both of which depend upon the average atomic number of the target.
# ! this correction is needed if there is a difference between the average atomic number of the specimen and that of the standard
# “light elements in a heavy matrix usually yield values which are too high.” ([Goldstein et al., 1981, p. 317])
avg_z_spec = C[0]*Z[0] + C[1]*Z[1]
print('Average atomic number of the specimen: ',avg_z_spec)
############################## for Cu Ka ############################## 
## R backscattering correction factors range from 0.5-1 (close to 1 at low atomic numbers) 
# varies with Z and overvoltage U = Eo/Ec (as decreases towards 1 few electrons are backscattered
U = E0/Ec[1]
R1 = 8.73 * 0.001 * pow(U,3) - 0.1669 * pow(U,2) + 0.9662 * U + 0.4523
R2 = 2.703 * 0.001 * pow(U,3) - 5.182 * 0.01 * pow(U,2) + 0.302*U -0.1836
R3 = (0.887 * pow(U,3) - 3.44 * U * U + 9.33 * U - 6.43)/pow(U,3)

R_Cu = R1 - R2 * math.log(R3*Z[0] + 25)
R_Al = R1 - R2 * math.log(R3*Z[1] + 25)

print("R Al, ", R_Al)
print("R Cu, ", R_Cu)
print("Cu Ka: ")
print("U, overvoltage", U)
print("R1, ", R1)
print("R2, ", R2)
print("R3, ", R3)

# Ri = sum(j) CjRij
# Ri = C[0] * Rij
# # mean ionization potential J = 9.76 * Z + 58.8 * pow(Z, -0.19) [eV] # Berger and Seltzer (1964)
J = [9.76 * Z[0] + 58.8 * pow(Z[0], -0.19), 9.76 * Z[1] + 58.8 * pow(Z[1], -0.19)]
print("J, ", J)
# # S is the electron stopping power 
# # “i represents the element i which is measured and j represents each of the elements present in the standard or specimen including element i.” ([Goldstein et al., 1981, p. 319]
S_Cu_Al = (Z[0] / (A[0]*(E0 + Ec[1]))) * math.log(583*(E0+Ec[1])/J[0])
S_Cu_Cu = (Z[1] / (A[1]*(E0 + Ec[1]))) * math.log(583*(E0+Ec[1])/J[1])
S_Cu = C[0]*S_Cu_Al + C[1] * S_Cu_Cu
print("S_Cu", S_Cu)
# Al
S_Al_Cu = (Z[1] / (A[1]*(E0 + Ec[0]))) * math.log(583*(E0+Ec[0])/J[1])
S_Al_Al = (Z[0] / (A[0]*(E0 + Ec[0]))) * math.log(583*(E0+Ec[0])/J[0])
S_Al = C[0]*S_Al_Al + C[1] * S_Al_Cu
print("S_Al", S_Al)
## For the standard
# Using a Cu standard
print("\n## For the standard:")
Si = (Z[1] / (A[1]*(E0 + Ec[1]))) * math.log(583*(E0+Ec[1])/J[1])
print("Si standard, ", Si)
# R Standard
U = E0/Ec[1]
R1 = 8.73 * 0.001 * pow(U,3) - 0.1669 * pow(U,2) + 0.9662 * U + 0.4523
R2 = 2.703 * 0.001 * pow(U,3) - 5.182 * 0.01 * pow(U,2) + 0.302*U -0.1836
R3 = (0.887 * pow(U,3) - 3.44 * U * U + 9.33 * U - 6.43)/pow(U,3)
Ri = R1 - R2 * math.log(R3*Z[1] + 25)
print("Ri standard, ", Ri)
## For the speciment
Ri_star = C[0] * R_Al + C[1] * R_Cu
Si_star = C[0] * S_Al + C[1] * S_Cu
print("\n## For the specimen:")
print("The Ri star is, ", Ri_star)
print("The Si star is, ", Si_star)
# # if the integration and the Zi equation is avoided and Q the cross section is constant, we have (Heinrich and Yakowitz, 1970)
# Zi = (Ri/Ri*) * (Si*/Si)

"""
As discussed earlier unless the atomic number effect is corrected for, 
analyses of heavy elements in a light element matrix (Cu in AI) generally 
yield values which are too low (Zj > I) while analyses of light elements 
in a heavy matrix (AI in Cu) usually yield values which are too high (Zj < I).
"""


Average atomic number of the specimen:  28.68
R Al,  0.9102364973019827
R Cu,  0.9681663709445439
Cu Ka: 
U, overvoltage 1.6703786191536747
R1,  1.6412286412601202
R2,  0.1888656935816759
R3,  0.7918338925748151
J,  [162.9983655505653, 314.05142382876625]
S_Cu 0.07257841326676809
S_Al 0.09488484963049472

## For the standard:
Si standard,  0.07223411799733445
Ri standard,  0.9102364973019827

## For the specimen:
The Ri star is,  0.9670077734716926
The Si star is,  0.07302454199404262


'\nAs discussed earlier unless the atomic number effect is corrected for, \nanalyses of heavy elements in a light element matrix (Cu in AI) generally \nyield values which are too low (Zj > I) while analyses of light elements \nin a heavy matrix (AI in Cu) usually yield values which are too high (Zj < I).\n'